# Necessary information
1. multispectral imagery with no thermal band and scaled to a pixel size that addresses field LAI measurement scales, e.g. whole canopy extension (several meters)
2. sensor response function for each shortwave band
3. Python libraries as requested in README.md
4. Make sure GDAL is adequately installed.

:::{note}
Using MicaSense cameras nomenclature:
https://support.micasense.com/hc/en-us/articles/214878778-What-is-the-center-wavelength-and-bandwidth-of-each-filter-
:::



# Import libraries
First we need to import all required libraries and set some constants pointing to the location of the spectral response functions and soil reflectance libraries

In [ ]:
# import necessary libraries
from pathlib import Path
import multiprocessing as mp
import numpy as np
import pandas as pd
import datetime as dt
from datetime import date
from datetime import datetime
from osgeo import gdal
import Py6S as sixs
from pyproj import Proj #added
from pypro4sail import machine_learning_regression as inv
from pyTSEB import meteo_utils as met
import sklearn 
from sklearn.ensemble import RandomForestRegressor as rf_sklearn
print("Imported all libraries")

# Dictionary that sets the band order of each UAV image
BANDS_DICT = {
    "Dual": ["B06", "B01", "B07", "B02", "B08","B03", "B09", "B05", "B10", "B04"],
    "Altum":  [ "B01", "B02","B03", "B05", "B04"]
    }

# Path to the soil spectral library
SOIL_LIBRARY = Path(inv.__file__).parent / "spectra" / "soil_spectral_library"
# Path to the spectral response functions
SRF_LIBRARY = Path(inv.__file__).parent / "spectra" / "sensor_response_functions"

# Default atmospheric values
DEFAULT_AOD = 0.1  # Aerosol Optical Thickness
DEFAULT_WVP = 1.0  # Total column water vapour

# Default observation geometry
MEAN_VZA = 0
MEAN_VAA = 0

# Pro4SAIL object parameter names
OBJ_PARAM_NAMES = ["Cab", "Car", "Cm", "Cw", "Ant", "Cbrown",
                   "LAI", "leaf_angle"]

# output folder
OUTPUT_FOLDER = Path() / "outputs"

# Set the input image and sensor type

In [ ]:
## Running ProSAIL simulations on a MicaSense 10 band sensor
image_name = "20220708_almond_ripperdan_fl7_1329_10band_refl"
layer_path = Path() / "inputs" / f"{image_name}.tif"

# scaling factor reflectance image
layer_scale = 1

sensor = "Dual"

srf_file = SRF_LIBRARY / "micasense.txt"


## Get acquisition date and time

In [ ]:
date_flight = dt.datetime.strptime(layer_path.stem.split("_")[0], "%Y%m%d")
time_flight = layer_path.stem.split("_")[4]
dec_time = float(time_flight[:2]) + float(time_flight[2:]) / 60
doy = int(date_flight.strftime("%j"))
print(f"UAV flight acquire on DOY {doy} at {dec_time:.2f} local time")


# Read the Spectral Response Function

In [ ]:
bands = BANDS_DICT[sensor]

# Stack spectral bands
srf = []

srfs = np.genfromtxt(srf_file, dtype=None, names=True)
for band in bands:
    srf.append(srfs[band])


# Define ancillary functions
We need two ancillary functions to run the 4SAIL simuations.

## Spectral diffuse radiation
This function will run 6S atmospheric RTM based on the acqusition time, solar geommetry and atmopsheric inforamtion to generate the spectral distritubion of diffuse radiation `skyl`

In [ ]:
# Function to run 6S and get diffuse radiation fraction
def get_diffuse_radiation_6S(aot, wvp, sza, saa, date,
                             altitude=0.1, wls_step=10, n_jobs=None):

    s = sixs.SixS()
    s.atmos_profile = sixs.AtmosProfile.PredefinedType(
                            sixs.AtmosProfile.MidlatitudeSummer)

    s.aeroprofile = sixs.AeroProfile.PredefinedType(
                            sixs.AeroProfile.Continental)
    
    s.ground_reflectance = sixs.GroundReflectance.HomogeneousLambertian(0)

    if np.isfinite(wvp) and wvp > 0:
        s.atmos_profile = sixs.AtmosProfile.UserWaterAndOzone(wvp, 0.9)
    
    if np.isfinite(aot) and aot > 0:
        s.aot550 = aot

    s.geometry.solar_z = sza
    s.geometry.solar_a = saa
    s.geometry.view_z = 0
    s.geometry.view_a = 0
    s.geometry.day = date.day
    s.geometry.month = date.month

    s.altitudes.set_target_custom_altitude(altitude)
    s.wavelength = sixs.Wavelength(0.4, 2.5)

    wls = np.arange(400, 2501)
    wls_sim = np.arange(400, 2501, wls_step)

    wv, res = sixs.SixSHelpers.Wavelengths.run_wavelengths(s,
                                                           wls_sim / 1000.,
                                                           verbose=False,
                                                           n=n_jobs)
    
    eg_d = np.array(sixs.SixSHelpers.Wavelengths.extract_output(res, 
                                                    'diffuse_solar_irradiance'))
    
    eg_s = np.array(sixs.SixSHelpers.Wavelengths.extract_output(res, 
                                                    'direct_solar_irradiance'))
    
    eg_d = np.maximum(eg_d, 0)
    eg_s = np.maximum(eg_s, 0)
    skyl = np.full_like(wls, np.nan, dtype=np.float64)
    # Fill the diffuse values into a full wavelenght array
    valid = np.in1d(wls, wls_sim, assume_unique=True)
    skyl[valid] = eg_d / (eg_d + eg_s)
    # Fill nans by linear interpolation
    nans, x = np.isnan(skyl), lambda z: z.nonzero()[0]
    skyl[nans] = np.interp(x(nans), x(~nans), skyl[~nans])    
    
    return skyl

## Soil spectral library
This second function, will a series ofe random soil reflectances based on the ECOSTRESS spectral library stored in [pypro4sail/spectra/soil_spectral_library](.[pypro4sail/spectra/soil_spectral_library)

In [ ]:
# Function to build soil database for Pro4SAIL simulations
def build_soil_database(soil_albedo_factor,
                        soil_library=SOIL_LIBRARY):

    soil_library = Path(soil_library)
    n_simulations = np.size(soil_albedo_factor)
    soil_files = list(soil_library.glob('jhu.*spectrum.txt'))
    n_soils = len(soil_files)
    soil_spectrum = []
    for soil_file in soil_files:
        r = np.genfromtxt(soil_file)
        soil_spectrum.append(r[:, 1])

    multiplier = int(np.ceil(float(n_simulations / n_soils)))
    soil_spectrum = np.asarray(soil_spectrum * multiplier)
    soil_spectrum = soil_spectrum[:n_simulations]
    soil_spectrum = soil_spectrum * soil_albedo_factor.reshape(-1, 1)
    soil_spectrum = np.clip(soil_spectrum, 0, 1)
    soil_spectrum = soil_spectrum.T
    return soil_spectrum

# Read image geometry

In [ ]:
# Get raster information
fid = gdal.Open(str(layer_path), gdal.GA_ReadOnly)
gt = fid.GetGeoTransform()
proj = fid.GetProjection()
x_size = fid.RasterXSize
y_size = fid.RasterYSize
bands = fid.RasterCount
lr_x = gt[0] + x_size * gt[1] + y_size * gt[2]
lr_y = gt[3] + x_size * gt[4] + y_size * gt[5]
extent = [gt[0], lr_y, lr_x, gt[3]]
del fid
center = np.mean([gt[0], lr_x]), np.mean([gt[3], lr_y])
p = Proj(proj)
center_geo = p(center[0], center[1], inverse=True)

# Get standard longitude
stdlon = round(center_geo[0]/15,0)*15


# Simulations with ProspectD+4SAIL
We first define the number of ProspectD+4SAIL Sobol' simulations and the number of CPUs to be used in parallel

In [ ]:
n_simulations = 20000
n_jobs = mp.cpu_count()

## create the random biophysical traits and soil spectra

In [ ]:
start_time = dt.datetime.today()
params_orig = inv.build_prosail_database(n_simulations,
                                         distribution=inv.SALTELLI_DIST)

print(f"Builing standard soil database")
soil_spectrum = build_soil_database(params_orig["bs"])

elapsed = (dt.datetime.today() - start_time).total_seconds()
print(f"ProspectD+4SAIL simulations run in {elapsed} seconds")

## create the spectral distribution of difuse radiation

In [ ]:
start_time = dt.datetime.today()

# Get Solar angles
sza, saa = met.calc_sun_angles(center_geo[1],
                               center_geo[0], 
                               stdlon,
                               doy, 
                               dec_time)

sza =float(sza)
saa =float(saa)

print(f"Running 6S for estimation of diffuse/direct irradiance")
skyl = get_diffuse_radiation_6S(DEFAULT_AOD, DEFAULT_WVP, sza, saa, date_flight,
                                altitude=0.1)

elapsed = (dt.datetime.today() - start_time).total_seconds()
print(f"6S diffuse radiation created in {elapsed} seconds")


## Run the vectorized version of ProspectD+4SAIL in parallel CPUs

In [ ]:
start_time = dt.datetime.today()

if "fAPAR" in OBJ_PARAM_NAMES or "fIPAR" in OBJ_PARAM_NAMES:
    calc_fapar = True
else:
    calc_fapar = False

rho_canopy_vec, params = inv.simulate_prosail_lut_parallel(
    n_jobs,
    params_orig,
    np.arange(400,2501),
    soil_spectrum,
    skyl=skyl,
    sza=sza,
    vza=MEAN_VZA,
    psi=saa - MEAN_VAA,
    srf=srf,
    outfile=None,
    calc_FAPAR=calc_fapar,
    reduce_4sail=True)

params = pd.DataFrame(params)
elapsed = (dt.datetime.today() - start_time).total_seconds()
print(f"ProspectD+4SAIL simulations run in {elapsed} seconds")

# Read the input reflectance bands

In [ ]:
start_time = dt.datetime.today()
fid = gdal.Open(str(layer_path), gdal.GA_ReadOnly)
image_array = []
for band in range(bands):
    image_array.append(fid.GetRasterBand(band + 1).ReadAsArray())
del fid
image_array = np.array(image_array)
dims = image_array.shape[1:]
image_array = image_array.reshape((image_array.shape[0], -1)).T
image_array = image_array / layer_scale
elapsed = (dt.datetime.today() - start_time).total_seconds()
print(f"Image reflectance factors read in {elapsed} seconds")

# Train and apply a Random Forests to the reflectance image
For each biophysical trait we will train a Random Forest with the simulated reflectance factors. Then this Random Forest will be applied to the input image top of the canopy reflectances to get the estimated trait

In [ ]:
start_time = dt.datetime.today()
scikit_regressor_opts = {"n_estimators": 100,
                         "min_samples_leaf": 1,
                         "n_jobs": n_jobs}

reg = rf_sklearn(**scikit_regressor_opts)

driver = gdal.GetDriverByName("GTiff")
for i, param in enumerate(OBJ_PARAM_NAMES):
    print(f'Applying {param} regression model to image')
    reg = reg.fit(rho_canopy_vec, params[param])
    output = reg.predict(image_array)
    output = output.reshape(dims)
    if param == 'fAPAR' or param == 'fIPAR':
        min_value = 0
        max_value = 1
    else:
        min_value = inv.prosail_bounds[param][0]
        max_value = inv.prosail_bounds[param][1]

    output = np.clip(output, min_value, max_value)
    # Save the outputs in single band GeoTIFF
    output_file = OUTPUT_FOLDER /  f"{image_name}_{param.upper()}.tif"
    print(f"Saving {param} in {output_file}")
    ds = driver.Create(output_file, dims[1], dims[0], 1, gdal.GDT_Float32)
    ds.SetProjection(proj)
    ds.SetGeoTransform(gt)
    ds.GetRasterBand(1).WriteArray(output)
    ds.FlushCache()
    del output

print(f"Biophysical traits created in {elapsed} seconds")